<a href="https://colab.research.google.com/github/reederickson/deep-learning-challenge/blob/main/AlphabetSoupCharity_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' not 'NAME'
application_df.drop(columns=["EIN"], inplace=True)

In [3]:
# Look at name value counts for binning
name_counts = application_df['NAME'].value_counts()
print(name_counts)

NAME
PARENT BOOSTER USA INC                                     1260
TOPS CLUB INC                                               765
UNITED STATES BOWLING CONGRESS INC                          700
WASHINGTON STATE UNIVERSITY                                 492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC             408
                                                           ... 
NEW MEXICO ASSOCIATION OF INDEPENDENT SCHOOLS INC             1
DC COUNCIL MINISTRIES                                         1
CHRISTIANS AGAINST GUN VIOLENCE ENDAGERING OUR CHILDREN       1
MCKINNEY CITIZENS FIRE ACADEMY ALUMNI ASSOCIATION             1
AIESEC ALUMNI IBEROAMERICA CORP                               1
Name: count, Length: 19568, dtype: int64


In [11]:
name_counts = application_df['NAME'].value_counts()
name_counts_greater_than_1 = name_counts[name_counts >100]
print(name_counts_greater_than_1)

NAME
PARENT BOOSTER USA INC                                                1260
TOPS CLUB INC                                                          765
UNITED STATES BOWLING CONGRESS INC                                     700
WASHINGTON STATE UNIVERSITY                                            492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC                        408
PTA TEXAS CONGRESS                                                     368
SOROPTIMIST INTERNATIONAL OF THE AMERICAS INC                          331
ALPHA PHI SIGMA                                                        313
TOASTMASTERS INTERNATIONAL                                             293
MOST WORSHIPFUL STRINGER FREE AND ACCEPTED MASONS                      287
LITTLE LEAGUE BASEBALL INC                                             277
INTERNATIONAL ASSOCIATION OF LIONS CLUBS                               266
MOMS CLUB                                                              210
INTERNATIONAL ASSOCI

In [12]:
# Choose a cutoff value and create a list of name to be replaced
# use the variable name `name_to_replace`
cutoff_value= 100
name_to_replace= application_df["NAME"].value_counts()[application_df['NAME'].value_counts()< cutoff_value].index
# Replace in dataframe
for name in name_to_replace:
    application_df['NAME'] = application_df['NAME'].replace(name,"Other")
# Drop rows with "Other" in the "NAME" column
application_df = application_df[application_df["NAME"] != "Other"]

# Check to make sure binning was successful
application_df['NAME'].value_counts()

NAME
PARENT BOOSTER USA INC                                                1260
TOPS CLUB INC                                                          765
UNITED STATES BOWLING CONGRESS INC                                     700
WASHINGTON STATE UNIVERSITY                                            492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC                        408
PTA TEXAS CONGRESS                                                     368
SOROPTIMIST INTERNATIONAL OF THE AMERICAS INC                          331
ALPHA PHI SIGMA                                                        313
TOASTMASTERS INTERNATIONAL                                             293
MOST WORSHIPFUL STRINGER FREE AND ACCEPTED MASONS                      287
LITTLE LEAGUE BASEBALL INC                                             277
INTERNATIONAL ASSOCIATION OF LIONS CLUBS                               266
MOMS CLUB                                                              210
INTERNATIONAL ASSOCI

In [13]:
# Look at ASK_AMT value counts for binning
ask_amt_counts = application_df['ASK_AMT'].value_counts()
print(ask_amt_counts)

ASK_AMT
5000      7387
32807        2
15039        2
59711        1
26039        1
          ... 
272119       1
40899        1
27376        1
5759         1
10196        1
Name: count, Length: 924, dtype: int64


In [14]:
# Choose a cutoff value and create a list of ask amount to be replaced
# use the variable name `ask_amt_to_replace`
cutoff_value= 5000
ask_amt_to_replace= application_df["ASK_AMT"].value_counts()[application_df['ASK_AMT'].value_counts()< cutoff_value].index
# Replace in dataframe
for ask in ask_amt_to_replace:
    application_df['ASK_AMT'] = application_df['ASK_AMT'].replace(ask,"Other")

# Check to make sure binning was successful
application_df['ASK_AMT'].value_counts()

ASK_AMT
5000     7387
Other     925
Name: count, dtype: int64

In [15]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
cutoff_value= 500
application_types_to_replace= application_df["APPLICATION_TYPE"].value_counts()[application_df['APPLICATION_TYPE'].value_counts()< cutoff_value].index
# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       6588
Other    1216
T4        508
Name: count, dtype: int64

In [16]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cutoff_value_cls= 600
classifications_to_replace = application_df['CLASSIFICATION'].value_counts()[application_df['CLASSIFICATION'].value_counts() < cutoff_value_cls].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    3065
C1200    1629
C2100    1452
C2000    1421
C3000     636
Other     109
Name: count, dtype: int64

In [17]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df= pd.get_dummies(application_df,dtype=float)
# application_df.head()

In [18]:
# Split our preprocessed data into our features and target arrays
y= application_df["IS_SUCCESSFUL"]
X= application_df.drop(columns=["IS_SUCCESSFUL"])

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [19]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Updated training model to increase performance

In [20]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  100
hidden_nodes_layer2 = 80
hidden_nodes_layer3 = 80
hidden_nodes_layer4 = 80

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               6300      
                                                                 
 dense_1 (Dense)             (None, 80)                8080      
                                                                 
 dense_2 (Dense)             (None, 80)                6480      
                                                                 
 dense_3 (Dense)             (None, 80)                6480      
                                                                 
 dense_4 (Dense)             (None, 1)                 81        
                                                                 
Total params: 27421 (107.11 KB)
Trainable params: 27421 (107.11 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## Compile, Train and Evaluate the Model

In [21]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [22]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=150)

Epoch 1/150
195/195 [==============================] - 3s 4ms/step - loss: 0.2671 - accuracy: 0.8994
Epoch 2/150
195/195 [==============================] - 1s 4ms/step - loss: 0.2167 - accuracy: 0.9169
Epoch 3/150
195/195 [==============================] - 1s 5ms/step - loss: 0.2121 - accuracy: 0.9174
Epoch 4/150
195/195 [==============================] - 1s 6ms/step - loss: 0.2103 - accuracy: 0.9203
Epoch 5/150
195/195 [==============================] - 1s 5ms/step - loss: 0.2077 - accuracy: 0.9209
Epoch 6/150
195/195 [==============================] - 1s 4ms/step - loss: 0.2069 - accuracy: 0.9190
Epoch 7/150
195/195 [==============================] - 1s 4ms/step - loss: 0.2063 - accuracy: 0.9216
Epoch 8/150
195/195 [==============================] - 1s 4ms/step - loss: 0.2052 - accuracy: 0.9212
Epoch 9/150
195/195 [==============================] - 1s 4ms/step - loss: 0.2046 - accuracy: 0.9216
Epoch 10/150
195/195 [==============================] - 1s 4ms/step - loss: 0.2049 - accura

In [23]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

65/65 - 1s - loss: 0.2813 - accuracy: 0.9143 - 693ms/epoch - 11ms/step
Loss: 0.28133079409599304, Accuracy: 0.9143407344818115


In [24]:
# Export our model to HDF5 file
optimized_model_filename= "AlphabetSoupCharityOptimization.h5"
nn.save(optimized_model_filename)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
